In [166]:
from os import listdir
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

def cal_mean(c):
    # calculate mean of 3 different tries
    return np.around(np.add.reduceat(c, np.arange(0, len(c), 3))/3,3)

def calculate_stats(path, roll_avg, truncate, info):
    dir_ = 'Data/'
    filenames = [filename for filename in listdir(dir_ + path) if filename.endswith('.csv')]
    
    for i in range(len(filenames)):
        file = filenames[i]
        title = file.split('_')[0]

        # extract dataframe
        df = pd.read_csv(dir_ + path + file)

        # combine triaxial data into one singular magnitude data
        if 'Accelerometer' in file:
            df['x'] = df.loc[roll_avg-1:,'x-axis (g)']-df['x-axis (g)'].rolling(roll_avg).mean()[roll_avg-1:]
            df['y'] = df.loc[roll_avg-1:,'y-axis (g)']-df['y-axis (g)'].rolling(roll_avg).mean()[roll_avg-1:]
            df['z'] = df.loc[roll_avg-1:,'z-axis (g)']-df['z-axis (g)'].rolling(roll_avg).mean()[roll_avg-1:]
            df['magnitude'] = df['x']**2 + df['y']**2 + df['z']**2
            df['magnitude'] = (df['magnitude'].apply(math.sqrt))*9.81

        elif 'Gyroscope' in file:
            df['x'] = df.loc[roll_avg-1:,'x-axis (deg/s)']-df['x-axis (deg/s)'].rolling(roll_avg).mean()[roll_avg-1:]
            df['y'] = df.loc[roll_avg-1:,'y-axis (deg/s)']-df['x-axis (deg/s)'].rolling(roll_avg).mean()[roll_avg-1:]
            df['z'] = df.loc[roll_avg-1:,'z-axis (deg/s)']-df['x-axis (deg/s)'].rolling(roll_avg).mean()[roll_avg-1:]
            df['magnitude'] = df['x']**2 + df['y']**2 + df['z']**2
            df['magnitude'] = df['magnitude'].apply(math.sqrt)

        # truncate signal beyond start and end times
        df = df[(df['elapsed (s)'] <= truncate[i][1]) & (df['elapsed (s)'] >= truncate[i][0])]

        # extract umbrella curve
        df['max'] = df['magnitude'].rolling(30).max()
        
        # calculate rolling frequency
        freqs, times, spectrogram = signal.spectrogram(df['magnitude'].values, fs=50.0, nperseg=70, noverlap=70-1)
        df_ = pd.DataFrame({'time':times, 'freq':[freqs[i] for i in np.argmax(spectrogram,0)]})
        df_['max'] = df_['freq'].rolling(30).max()

        # gather information on amplitude
        stats = df.describe().loc[:,['max','magnitude']].T
        # gather information on frequency
        stats_ = df_.describe().loc[:,['max','freq']].T
        
        # append into dataframe
        info = info.append({
            'test': title,
            'mean_amp': stats['mean'][0].round(3),
            'std_amp': stats['std'][1].round(3),
            'mean_freq': stats_['mean'][0].round(3),
            'std_freq': stats_['std'][1].round(3)
        }, ignore_index=True)
        
    return info


info = pd.DataFrame(columns=['test','mean','std'])

# HANDGRIP
truncate = [(80,107),(80,107),(12.5,39.5),(12.5,39.5),(10,42),(10,42),(46,78),(46,78),(52,92),(52,92),(30,70),(30,70),
            (50,105),(50,105),(14,69),(14,69),(13,68),(13,68),(70,125),(70,125),(33.5,68.5),(33.5,68.5),(5,40),(5,40)]
info = calculate_stats('handgrip_experiment/', 3, truncate, info)

# PRONATION AND SUPINATION
truncate = [(127.5,160),(127.5,160),(196,229),(196,229),(48.5,80),(48.5,80),(17.5,50),(17.5,50),(15,51),(15,51),(50,86),(50,86),
            (44,80),(44,80),(63,101),(63,101),(9,48),(9,48),(27,65),(27,65),(10,60),(10,60),(60,107),(60,107)]
info = calculate_stats('pronationsupination_experiment/', 3, truncate, info)

# REARRANGE DATAFRAME
info['name'] = info['test'].apply(lambda x:x.split('-')[0])
info['level'] = info['test'].apply(lambda x:x.split('-')[1])
info['try'] = info['test'].apply(lambda x:x.split('-')[2])

info = pd.DataFrame({'name': info['name'].iloc[::2].values,
                     'level': info['level'].iloc[::2].apply(lambda x:int(x[-1])).values,
                     'try': info['try'].iloc[::2].apply(lambda x:int(x[-1])).values,
                     'mean_accel_amp': info['mean_amp'].iloc[::2].values,
                     'mean_gyro_amp': info['mean_amp'].iloc[1::2].values,
                     'std_accel_amp': info['std_amp'].iloc[::2].values,
                     'std_gyro_amp': info['std_amp'].iloc[1::2].values,
                     'mean_accel_freq': info['mean_freq'].iloc[::2].values,
                     'mean_gyro_freq': info['mean_freq'].iloc[1::2].values,
                     'std_accel_freq': info['std_freq'].iloc[::2].values,
                     'std_gyro_freq': info['std_freq'].iloc[1::2].values})

info = pd.DataFrame({'name': info['name'].iloc[::2].values,
                     'level': info['level'].iloc[::2].values,
                     'try': info['try'].iloc[::2].values,
                     'mean_acc_forearm_amp': info['mean_accel_amp'].iloc[::2].values,
                     'mean_acc_wrist_amp': info['mean_accel_amp'].iloc[1::2].values,
                     'mean_gyr_forearm_amp': info['mean_gyro_amp'].iloc[::2].values,
                     'mean_gyr_wrist_amp': info['mean_gyro_amp'].iloc[1::2].values,
                     'std_acc_forearm_amp': info['std_accel_amp'].iloc[::2].values,
                     'std_acc_wrist_amp': info['std_accel_amp'].iloc[1::2].values,
                     'std_gyr_forearm_amp': info['std_gyro_amp'].iloc[::2].values,
                     'std_gyr_wrist_amp': info['std_gyro_amp'].iloc[1::2].values,
                     'mean_acc_forearm_freq': info['mean_accel_freq'].iloc[::2].values,
                     'mean_acc_wrist_freq': info['mean_accel_freq'].iloc[1::2].values,
                     'mean_gyr_forearm_freq': info['mean_gyro_freq'].iloc[::2].values,
                     'mean_gyr_wrist_freq': info['mean_gyro_freq'].iloc[1::2].values,
                     'std_acc_forearm_freq': info['std_accel_freq'].iloc[::2].values,
                     'std_acc_wrist_freq': info['std_accel_freq'].iloc[1::2].values,
                     'std_gyr_forearm_freq': info['std_gyro_freq'].iloc[::2].values,
                     'std_gyr_wrist_freq': info['std_gyro_freq'].iloc[1::2].values})

info = pd.DataFrame({'name': info['name'][::3].values,
                     'level': info['level'][::3].values,
                     'mean_acc_forearm_amp': cal_mean(info['mean_acc_forearm_amp'].values),
                     'mean_acc_wrist_amp': cal_mean(info['mean_acc_wrist_amp'].values),
                     'mean_gyr_forearm_amp': cal_mean(info['mean_gyr_forearm_amp'].values),
                     'mean_gyr_wrist_amp': cal_mean(info['mean_gyr_wrist_amp'].values),
                     'std_acc_forearm_amp': cal_mean(info['std_acc_forearm_amp'].values),
                     'std_acc_wrist_amp': cal_mean(info['std_acc_wrist_amp'].values),
                     'std_gyr_forearm_amp': cal_mean(info['std_gyr_forearm_amp'].values),
                     'std_gyr_wrist_amp': cal_mean(info['std_gyr_wrist_amp'].values),
                     'mean_acc_forearm_freq': cal_mean(info['mean_acc_forearm_freq'].values),
                     'mean_acc_wrist_freq': cal_mean(info['mean_acc_wrist_freq'].values),
                     'mean_gyr_forearm_freq': cal_mean(info['mean_gyr_forearm_freq'].values),
                     'mean_gyr_wrist_freq': cal_mean(info['mean_gyr_wrist_freq'].values),
                     'std_acc_forearm_freq': cal_mean(info['std_acc_forearm_freq'].values),
                     'std_acc_wrist_freq': cal_mean(info['std_acc_wrist_freq'].values),
                     'std_gyr_forearm_freq': cal_mean(info['std_gyr_forearm_freq'].values),
                     'std_gyr_wrist_freq': cal_mean(info['std_gyr_wrist_freq'].values)})

info

,name,level,mean_acc_forearm_amp,mean_acc_wrist_amp,mean_gyr_forearm_amp,mean_gyr_wrist_amp,std_acc_forearm_amp,std_acc_wrist_amp,std_gyr_forearm_amp,std_gyr_wrist_amp,mean_acc_forearm_freq,mean_acc_wrist_freq,mean_gyr_forearm_freq,mean_gyr_wrist_freq,std_acc_forearm_freq,std_acc_wrist_freq,std_gyr_forearm_freq,std_gyr_wrist_freq
0,hgrip,0,3.283,6.748,91.840,124.690,0.923,1.796,21.264,31.271,3.660,3.467,2.981,4.247,1.025,1.255,0.900,0.818
1,hgrip,4,0.894,1.865,17.809,28.179,0.315,0.713,5.397,9.030,4.690,6.350,4.567,4.837,4.683,6.084,3.251,2.988
2,prosup,0,2.116,2.672,109.981,188.474,0.536,0.717,30.433,58.228,6.435,2.710,2.774,2.237,4.577,1.131,0.737,0.389
3,prosup,4,1.269,2.219,37.408,84.132,0.390,0.711,11.267,26.389,8.785,5.599,4.848,4.727,6.520,4.598,2.851,2.594


In [167]:
info.T.to_excel('excel.xlsx')